# Determine the target sparsity in H analytically
This way, we can set initial parameters in a more sensible way.

In [137]:
import numpy as np
from scipy.stats import norm
from scipy.signal import convolve2d
np.set_printoptions(precision=4, suppress=True)

rho = 0.005
M = 15
seqLength = M
D = np.zeros((4, seqLength))
for i in range(seqLength):
    D[:,i] = np.random.multinomial(n=1, pvals=[0.25]*4)

iterations = 1000
totalSum = 0.
for i in range(iterations):
    test = np.random.randn(4, M)
    b = np.zeros(1) + norm.ppf(rho, 0, np.sqrt(M))
    C = convolve2d(D, test[::-1,::-1], mode='valid')
    C = C + b
    totalSum += np.sum(C > 0) / float(np.prod(C.shape))
print "Proportion of elements in H > 0: " +str(totalSum / iterations)

Proportion of elements in H > 0: 0.009


In [149]:
rho = 0.005
M = 9
print "rho: " + str(rho)
test = np.random.randn(4, M)
b = np.zeros(1) + scipy.stats.norm.ppf(rho, 0, np.sqrt(M))
print "Move mean by: " + str(b[0])
print "Percent of population > 0: " + str(float(np.sum(test > 0)) / float(np.prod(test.shape)))
print "Percent of population + bias > 0: " + str(float(np.sum((test+b) > 0)) / float(np.prod(test.shape)))

seqLength = 20000
D = np.zeros((4, seqLength))
for i in range(seqLength):
    D[:,i] = np.random.multinomial(n=1, pvals=[0.25]*4)

C = scipy.signal.convolve2d(D[::-1,::-1], test, mode='valid')
print "Result from conv"

print "Percent of H (before bias) > 0: " + str(np.sum(C > 0) / float(np.prod(C.shape)))
C = C + b
print "Percent of H (before pooling) > 0: " + str(np.sum(C > 0) / float(np.prod(C.shape)))

rho: 0.005
Move mean by: -7.72748791065
Percent of population > 0: 0.555555555556
Percent of population + bias > 0: 0.0
Result from conv
Percent of H (before bias) > 0: 0.632452981192
Percent of H (before pooling) > 0: 0.00535214085634


In [150]:
C_e = np.exp(C)
noAct = 1./ (1 + C_e)
act = C_e / (1 + C_e)

print noAct[0,0]
print act[0,0]
pos = noAct - act
print "Proportion where noact is greater than act: " + str(np.sum(pos > 0) / float(np.prod(C.shape)))
multinomial = np.concatenate((noAct, act), axis=0)

sample = np.zeros_like(multinomial)
for i in range(multinomial.shape[1]):
    s = np.random.multinomial(n=1, pvals=multinomial[:,i])
    sample[np.argmax(s), i] = 1

print "Percent of samples that activate: " + str(np.sum(sample[1,:]) / sample.shape[1])

0.999939352482
6.06475178583e-05
Proportion where noact is greater than act: 0.994647859144
Percent of samples that activate: 0.0235594237695
